# Quantum-Classical Hybrid Neural Network
This notebook implements a quantum-classical hybrid neural network to classify entries in the KDDCup dataset as 'good' or 'bad'.

## Steps in the Notebook:
1. Install necessary packages.
2. Load and preprocess the KDDCup dataset.
3. Define a quantum circuit as part of a hybrid quantum-classical neural network.
4. Train and evaluate the model.


### Introduction to Qiskit and Hybrid Models
**Qiskit** is a Python-based open-source framework for quantum computing. It provides tools to:
- Design and simulate quantum circuits.
- Execute quantum computations on real quantum devices and simulators.
- Integrate quantum operations into classical workflows.

This project uses Qiskit's Aer simulator, a high-performance simulator for running quantum circuits on classical hardware. 

**Key Quantum Concepts in This Notebook:**
1. **Parameterized Quantum Circuits:**
   - Quantum circuits with adjustable parameters, such as angles for rotation gates (`RY`).
   - Parameters are optimized during training to improve classification accuracy.
2. **Measurement:**
   - Extracting classical probabilities from quantum states by measuring qubits.
   - These probabilities are used in the hybrid neural network's forward pass.

By combining Qiskit with PyTorch, this notebook demonstrates a quantum-classical hybrid approach to solving machine learning problems.

In [ ]:
# Install specific versions of Qiskit and Qiskit-Aer
!pip install 'qiskit==0.41.0'
!pip install 'qiskit-aer==0.11.2'

### Load the KDDCup Dataset
The dataset is loaded and preprocessed for binary classification tasks.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from qiskit.providers.aer import Aer
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit, transpile, assemble
import numpy as np

# Load the dataset
df = pd.read_csv('kddcup.csv')

# Preprocess the dataset
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])
X = df.drop('class', axis=1).values
y = df['class'].values

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Convert to PyTorch datasets
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Define the Quantum Circuit
The quantum circuit uses parameterized gates (`RY`) and acts as a layer in the hybrid neural network.

In [ ]:
class QuantumCircuitWrapper:
    def __init__(self, n_qubits, backend, shots):
        self.n_qubits = n_qubits
        self.backend = backend
        self.shots = shots
        self.params = [Parameter(f"theta_{i}") for i in range(n_qubits)]
        self.qc = QuantumCircuit(n_qubits)

        for i in range(n_qubits):
            self.qc.h(i)
            self.qc.ry(self.params[i], i)

        self.qc.measure_all()

    def run(self, inputs):
        results = []
        for input_set in inputs:
            parameterized_circuit = self.qc.bind_parameters({param: theta for param, theta in zip(self.params, input_set)})
            qobj = assemble(transpile(parameterized_circuit, self.backend), shots=self.shots)
            job = self.backend.run(qobj)
            counts = job.result().get_counts()
            probabilities = np.array([counts.get(bin(i)[2:].zfill(self.n_qubits), 0) for i in range(2**self.n_qubits)]) / self.shots
            expectation = np.dot(probabilities, np.arange(2**self.n_qubits))
            results.append(expectation)
        return torch.tensor(results, dtype=torch.float32).view(-1, 1)

### Define the Hybrid Quantum-Classical Neural Network
The hybrid network combines classical fully connected layers with a quantum circuit layer.

In [ ]:
class HybridFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, inputs, quantum_circuit):
        ctx.quantum_circuit = quantum_circuit
        inputs_np = inputs.detach().numpy()
        expectations = quantum_circuit.run(inputs_np)
        ctx.save_for_backward(inputs)
        return expectations

    @staticmethod
    def backward(ctx, grad_output):
        inputs, = ctx.saved_tensors
        grad_input = grad_output.clone()
        return grad_input, None

class HybridLayer(nn.Module):
    def __init__(self, quantum_circuit):
        super(HybridLayer, self).__init__()
        self.quantum_circuit = quantum_circuit

    def forward(self, inputs):
        return HybridFunction.apply(inputs, self.quantum_circuit)

class QuantumHybridNet(nn.Module):
    def __init__(self):
        super(QuantumHybridNet, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 4)
        self.fc2 = nn.Linear(4, 2)
        self.hybrid = HybridLayer(QuantumCircuitWrapper(n_qubits=2, backend=Aer.get_backend('aer_simulator'), shots=100))
        self.fc3 = nn.Linear(1, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.hybrid(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

### Train and Evaluate the Model
The model is trained using the training dataset and evaluated on the test dataset.

In [ ]:
model = QuantumHybridNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.NLLLoss()

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        pred = output.argmax(dim=1)
        correct += (pred == y_batch).sum().item()
        total += y_batch.size(0)
print(f"Test Accuracy: {correct / total * 100:.2f}%")